In [4]:
from datasets import load_dataset

# Load the SQuAD 2.0 dataset
dataset = load_dataset("squad_v2")


Found cached dataset parquet (/Users/mousumi/.cache/huggingface/datasets/parquet/squad_v2-ea29cd02ee439285/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})


In [6]:
# from datasets import load_dataset

# # Load the dataset (assuming you already have it loaded as `dataset`)
# dataset = load_dataset("squad_v2")

# def extract_start_positions(batch):
#     # Initialize the list for start positions
#     start_positions = []

#     # Process each answer in the batch
#     for answers in batch['answers']:
#         # Check if 'answer_start' exists and is not empty
#         if 'answer_start' in answers and len(answers['answer_start']) > 0:
#             start_positions.append(answers['answer_start'][0])
#         else:
#             # Append a default value or handle the case where no start position is available
#             start_positions.append(None)  # or use a specific marker like -1

#     # Return the updated batch with new start_positions key
#     return {'start_positions': start_positions}

# # Apply the function to the dataset
# tokenized_datasets = dataset.map(extract_start_positions, batched=True, remove_columns=dataset["train"].column_names)

# # Example of accessing the modified dataset
# print(tokenized_datasets['train']['start_positions'][:10])


In [7]:
#!pip3 install --pre torch torchvision --extra-index-url https://download.pytorch.org/whl/nightly/cpu

In [8]:
#check for gpu
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [9]:
device = "mps" if torch.has_mps else "cpu"
print(f'Using device: {device}')

Using device: mps


In [10]:
import torch
torch.has_mps

True

In [11]:
# GPU
import time
start_time = time.time()

# syncrocnize time with cpu, otherwise only time for oflaoding data to gpu would be measured
#torch.mps.synchronize()

a = torch.ones(4000,4000, device="mps")
for _ in range(200):
    a +=a

elapsed_time = time.time() - start_time
print( "GPU Time: ", elapsed_time)

GPU Time:  0.16701412200927734


In [12]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizerFast, AdamW, get_linear_schedule_with_warmup

# Load the tokenizer and model
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Setup device
device = "mps" if torch.has_mps else "cpu"
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to the appropriate device


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [13]:
device

'mps'

In [14]:
def tokenize_squad_examples(examples):
    # Encode the inputs using batched=True to handle large datasets efficiently
    tokenized_inputs = tokenizer(
        examples['question'],
        examples['context'],
        max_length=384,
        truncation=True,
        padding="max_length",
        return_offsets_mapping=True
    )

    start_positions = []
    end_positions = []

    for i, offset_mapping in enumerate(tokenized_inputs["offset_mapping"]):
        answer = examples['answers'][i]
        start_char = answer['answer_start'][0]
        answer_length = len(answer['text'][0])
        
        sequence_ids = tokenized_inputs.sequence_ids(i)

        # Find the start index
        start_index = 0
        while sequence_ids[start_index] != 1:
            start_index += 1
        while start_index < len(offset_mapping) and (offset_mapping[start_index][0] <= start_char and offset_mapping[start_index][1] <= start_char):
            start_index += 1
        
        # Ensure start_index is within range
        if start_index >= len(offset_mapping):
            start_index = len(offset_mapping) - 1

        # Find the end index based on the length of the answer
        end_index = start_index
        while end_index < len(offset_mapping) and (offset_mapping[end_index][1] - offset_mapping[start_index][0] < answer_length):
            end_index += 1

        # Ensure end_index is within range
        end_index = min(end_index, len(offset_mapping) - 1)

        start_positions.append(start_index)
        end_positions.append(end_index)

    tokenized_inputs["start_positions"] = start_positions
    tokenized_inputs["end_positions"] = end_positions

    # Remove the offset mapping to avoid saving it during training
    del tokenized_inputs["offset_mapping"]
    
    return tokenized_inputs


In [15]:
##
from transformers import BertTokenizerFast
from datasets import load_dataset

# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def tokenize_and_align_labels(batch):
    # Tokenize the text
    tokenized_inputs = tokenizer(
        batch['question'],
        batch['context'],
        truncation=True,
        padding="max_length",
        max_length=384,
        return_offsets_mapping=True
    )

    start_positions = []
    end_positions = []

    # Iterate over each example in the batch
    for i in range(len(batch['question'])):
        # Check if 'answer_start' and 'text' are available
        if 'answer_start' in batch['answers'][i] and len(batch['answers'][i]['answer_start']) > 0:
            start_char = batch['answers'][i]['answer_start'][0]
            answer_text = batch['answers'][i]['text'][0]
        else:
            # Default values if data is missing
            start_char = 0
            answer_text = ""

        # Find the position of the answer within the tokenized context
        offset_mapping = tokenized_inputs['offset_mapping'][i]

        # Attempt to find the start token index
        start_token_index = next((idx for idx, offset in enumerate(offset_mapping) if start_char >= offset[0] and start_char < offset[1]), None)

        # Find the end token index based on the length of the answer
        end_token_index = start_token_index + len(tokenizer.encode(answer_text, add_special_tokens=False)) - 2 if start_token_index is not None else None

        start_positions.append(start_token_index if start_token_index is not None else 0)
        end_positions.append(end_token_index if end_token_index is not None else 0)

    # Save these positions back into the tokenized_inputs
    tokenized_inputs['start_positions'] = start_positions
    tokenized_inputs['end_positions'] = end_positions

    return tokenized_inputs

# Load the dataset and apply the tokenization
dataset = load_dataset("squad_v2")
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True, remove_columns=dataset["train"].column_names)

# Optional: show a preview
print(tokenized_datasets['train']['start_positions'][:3])
print(tokenized_datasets['train']['end_positions'][:3])



Found cached dataset parquet (/Users/mousumi/.cache/huggingface/datasets/parquet/squad_v2-ea29cd02ee439285/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/mousumi/.cache/huggingface/datasets/parquet/squad_v2-ea29cd02ee439285/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-cc819505d5d56daf.arrow
Loading cached processed dataset at /Users/mousumi/.cache/huggingface/datasets/parquet/squad_v2-ea29cd02ee439285/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-b6d27f9939969ad7.arrow


[75, 68, 143]
[77, 69, 142]


In [16]:
from torch.utils.data import DataLoader, TensorDataset
import torch
from datasets import load_dataset, DatasetDict

# Function to convert dataset features to tensors
def convert_to_tensors(dataset):
    # Convert lists of examples into tensors for each feature
    all_input_ids = torch.tensor(dataset['input_ids'], dtype=torch.long)
    all_attention_mask = torch.tensor(dataset['attention_mask'], dtype=torch.long)
    all_token_type_ids = torch.tensor(dataset['token_type_ids'], dtype=torch.long)
    all_start_positions = torch.tensor(dataset['start_positions'], dtype=torch.long)
    all_end_positions = torch.tensor(dataset['end_positions'], dtype=torch.long)

    return TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_start_positions, all_end_positions)

# Assuming 'tokenized_datasets' is a DatasetDict containing processed data
# Load your dataset (only necessary if not already loaded and processed)
# dataset = load_dataset("squad_v2")
# tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True, remove_columns=dataset["train"].column_names)

# Convert the processed dataset into tensors
train_dataset = convert_to_tensors(tokenized_datasets['train'])
validation_dataset = convert_to_tensors(tokenized_datasets['validation'])

# Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=8)

# Optionally print some information to confirm setup
print(f"Number of training batches: {len(train_loader)}")
print(f"Number of validation batches: {len(validation_loader)}")


Number of training batches: 16290
Number of validation batches: 1485


In [ ]:
import torch
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from transformers import BertForQuestionAnswering

# Load your model
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Assume train_loader is defined somewhere else, as provided in previous responses
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)  # Ensure this matches your dataset setup

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)  # Adjust learning rate as needed

# Compute the total number of training steps based on your data loader and desired number of epochs
num_epochs = 2  # Adjust number of epochs here
num_training_steps = len(train_loader) * num_epochs

# Setup the learning rate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=num_training_steps // 10,  # Usually 10% of train steps
    num_training_steps=num_training_steps
)

# Sample training loop skeleton (for completeness)
model.train()  # Set the model to training mode
for epoch in range(num_epochs):
    for batch in train_loader:
        inputs = {
            'input_ids': batch[0].to(model.device), 
            'attention_mask': batch[1].to(model.device),
            'start_positions': batch[3].to(model.device),
            'end_positions': batch[4].to(model.device)
        }
        outputs = model(**inputs)
        loss = outputs.loss

        loss.backward()  # Compute gradients
        optimizer.step()  # Update weights
        lr_scheduler.step()  # Update learning rate
        optimizer.zero_grad()  # Clear gradients for next batch

        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")  # Output loss for tracking progress


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

Epoch 1, Loss: 5.897670269012451
Epoch 1, Loss: 5.883021354675293
Epoch 1, Loss: 5.9113054275512695
Epoch 1, Loss: 5.867959976196289
Epoch 1, Loss: 5.958220481872559
Epoch 1, Loss: 5.816848278045654
Epoch 1, Loss: 5.670872688293457
Epoch 1, Loss: 5.788008689880371
Epoch 1, Loss: 5.991209983825684
Epoch 1, Loss: 5.904509544372559
Epoch 1, Loss: 5.886578559875488
Epoch 1, Loss: 5.957003116607666
Epoch 1, Loss: 5.867313385009766
Epoch 1, Loss: 5.788227081298828
Epoch 1, Loss: 5.830669403076172
Epoch 1, Loss: 5.775439262390137
Epoch 1, Loss: 5.663948059082031
Epoch 1, Loss: 5.913341045379639
Epoch 1, Loss: 5.813055515289307
Epoch 1, Loss: 5.7201714515686035
Epoch 1, Loss: 5.839244842529297
Epoch 1, Loss: 5.769522666931152
Epoch 1, Loss: 5.927454471588135
Epoch 1, Loss: 5.761710166931152
Epoch 1, Loss: 5.806033611297607
Epoch 1, Loss: 5.905063629150391
Epoch 1, Loss: 5.836753845214844
Epoch 1, Loss: 5.794981002807617
Epoch 1, Loss: 5.871476173400879
Epoch 1, Loss: 5.837308883666992
Epoch 1,

In [15]:
from torch.utils.data import DataLoader, TensorDataset
import torch

def convert_to_tensors(features):
    # Convert lists of examples into tensors for each feature
    all_input_ids = torch.tensor([f['input_ids'] for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f['attention_mask'] for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f['token_type_ids'] for f in features], dtype=torch.long)
    all_start_positions = torch.tensor([f['start_positions'] for f in features], dtype=torch.long)
    all_end_positions = torch.tensor([f['end_positions'] for f in features], dtype=torch.long)

    return TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_start_positions, all_end_positions)

# Assuming you have a function or method that produces tokenized_datasets
# For example:
# train_features = tokenize_squad_examples(train_examples)
# validation_features = tokenize_squad_examples(validation_examples)

train_dataset = convert_to_tensors(tokenized_datasets['train'])
validation_dataset = convert_to_tensors(tokenized_datasets['validation'])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=8)


In [2]:
# Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)  # Adjust learning rate as needed

# Compute the total number of training steps
num_training_steps = len(train_loader) * 2  # Assuming 3 epochs, adjust as needed

# Setup the learning rate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=num_training_steps // 10,  # Usually 10% of train steps
    num_training_steps=num_training_steps
)


NameError: name 'AdamW' is not defined

In [1]:
# Training loop
model.train()
for epoch in range(1):  # Adjust the number of epochs if needed
    total_train_loss = 0
    for batch in train_loader:
        input_ids, attention_mask, token_type_ids, start_positions, end_positions = [b.to(device) for b in batch]

        optimizer.zero_grad()

        outputs = model(input_ids, 
                        attention_mask=attention_mask, 
                        token_type_ids=token_type_ids, 
                        start_positions=start_positions, 
                        end_positions=end_positions)

        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()

        optimizer.step()
        lr_scheduler.step()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Epoch {epoch + 1} average training loss: {avg_train_loss}")


NameError: name 'model' is not defined